# Import

In [1]:
MOUNT_POINT = '/content/drive'
from google.colab import drive
drive.mount(MOUNT_POINT, force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
import torchvision.models as models
import torchvision
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hàm load dữ liệu

In [3]:
def load_data(path, test=True, r=1, agents=1):
    if test:
        X = pd.read_csv(path)
    else:
        path = path + "PC" + str(agents) + ".csv"
        X = pd.read_csv(path)

    Y = X.iloc[:, -1:]
    num_records = Y.shape[0]
    X = tf.constant(X.values[:, 0:785])
    #X = tf.reshape(X, (num_records, 28, 28, 1))  # Correct reshape to (num_records, 28, 28, 1)
    Y = tf.one_hot(Y.values[:, 0:], 2)
    Y = tf.reshape(Y, (num_records, 2))
    return X, Y

In [4]:
path = "/content/drive/MyDrive/SwarmLearn/COVID19/COVID-19_Radiography_Dataset/test.csv"
X_test, Y_test = load_data(path)

# Hàm đánh giá

In [5]:
def Evaluation_Function(global_model,X_test,y_test):
    y_predict = global_model.predict(X_test).argmax(1)
    y_true = np.array(y_test).argmax(1)
    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_true, y_predict)
    acc = acc.result().numpy()
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_predict)
    recall = recall.result().numpy()
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_predict)
    precision = precision.result().numpy()
    f_score =  2 * recall * precision / (recall + precision)
    return [acc,recall,precision,f_score]

# Model

In [6]:
def Model(X_test):
    input = tf.keras.layers.Input(shape=X_test.shape[1:])
    x = tf.keras.layers.Dense(64, activation='relu')(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(2, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=input, outputs=output)
    return model

In [7]:
from keras.optimizers import SGD
from sklearn.metrics import precision_score, recall_score, f1_score
model = Model(X_test)
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=["accuracy"]
)

# FL

In [18]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/Covid/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer='adam',
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 64, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"]
          )
      for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
      global_model.set_weights(avg_weights)

  print('Final Evaluating:')
  [acc,recall,precision,f_score] = Evaluation_Function(global_model, X_test, Y_test)
  return acc,recall,precision,f_score

In [19]:
FL_Train(rounds=10,agents=5,global_model=model)

Round  1
Local 0 Training:
Epoch 1/5
72/72 [==============================] - 3s 7ms/step - loss: 0.4717 - accuracy: 0.8198
Epoch 2/5
72/72 [==============================] - 1s 7ms/step - loss: 0.3646 - accuracy: 0.8737
Epoch 3/5
72/72 [==============================] - 1s 7ms/step - loss: 0.2481 - accuracy: 0.9136
Epoch 4/5
72/72 [==============================] - 1s 7ms/step - loss: 0.1562 - accuracy: 0.9481
Epoch 5/5
72/72 [==============================] - 0s 6ms/step - loss: 0.0981 - accuracy: 0.9708
Local 1 Training:
Epoch 1/5
72/72 [==============================] - 3s 5ms/step - loss: 0.4555 - accuracy: 0.8322
Epoch 2/5
72/72 [==============================] - 0s 5ms/step - loss: 0.3671 - accuracy: 0.8706
Epoch 3/5
72/72 [==============================] - 0s 5ms/step - loss: 0.2432 - accuracy: 0.9183
Epoch 4/5
72/72 [==============================] - 0s 5ms/step - loss: 0.1500 - accuracy: 0.9539
Epoch 5/5
72/72 [==============================] - 0s 5ms/step - loss: 0.0987 - ac

(0.9997803, 1.0, 0.9997465, 0.9998732048531483)